# Merging Market and News Data

This notebook merges stock market data from Intrinio, and news data from Thomson Reuters. The author thanks Robert Lutostanski, for getting access to the data.  

Author: Xin Wei (weixin0127@gmail.com), Indiana University Bloomington  
Version: 2019/12/31

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time

from pytz import UTC, timezone
from sklearn.preprocessing import MultiLabelBinarizer

# Display options
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)

## 1. Import Data

In [2]:
print("Importing data... This may take a few minutes.")
market_df = pd.read_csv("../Data/market_train_df.csv")
news_df = pd.read_csv("../Data/news_train_df.csv")
print("Importing data: done!")

Importing data... This may take a few minutes.
Importing data: done!


In [3]:
market_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0


In [4]:
news_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'EMRG', 'RTRS', 'CRU', 'ENR', 'ASIA', 'LEN', ...","{'Z', 'O', 'OIL'}",1438,1,NaN,False,11,275,"{'0857.HK', 'PTR.N', '0857.F', '0857.DE'}",PetroChina Co Ltd,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'CO', 'HT', 'LIF', 'LATAM', 'RTRS', 'MX', 'AS...","{'ESN', 'RNP', 'G', 'E', 'U', 'PCO', 'PCU', 'M...",4413,1,FEATURE,False,55,907,{'STA.N'},Travelers Companies Inc,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'BG', 'TH', 'RET', 'RTRS', 'ID', 'RO', 'ENR',...","{'M', 'RNP', 'D', 'PTD', 'E', 'U', 'T', 'PMF',...",2108,2,PRESS DIGEST,False,15,388,"{'WMT.N', 'WMT.DE'}",Wal-Mart Stores Inc,14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'LEI', 'INS', 'RTRS', 'USC', 'WWW', 'FUND', '...","{'M', 'RNP', 'D', 'PTD', 'E', 'U', 'T', 'PMF',...",1776,6,PRESS DIGEST,False,14,325,"{'GOOG.O', 'GOOG.OQ', 'GOOGa.DE'}",Google Inc,13,0.149071,-1,0.752917,0.162715,0.084368,83,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'LEI', 'INS', 'RTRS', 'USC', 'WWW', 'FUND', '...","{'M', 'RNP', 'D', 'PTD', 'E', 'U', 'T', 'PMF',...",1776,6,PRESS DIGEST,False,14,325,{'XMSR.O'},XM Satellite Radio Holdings Inc,11,0.149071,-1,0.699274,0.209360,0.091367,102,0,0,0,0,0,0,0,0,0,0


## 2. Preprocess and Merge the Two Datasets

In [5]:
class PreprocessAndMerge():
    """
    Preprocess market_data and news_data, merge the two datasets and save to out_filepath
    """
    def __init__(self, market_df, news_df, out_filepath):
        """ Pass inputs to class variables
        """
        self.market_data = market_df
        self.news_data = news_df
        self.out_filepath = out_filepath

    def find_asset_name_map(self, market_data, news_data):
        """ Find assetName correspondences between market_data and news_data
        """
        # Get assetCode and assetName for both market_data and news_data
        code_name_market = market_data[['assetCode', 'assetName']].drop_duplicates(subset=['assetCode'])
        code_name_news = news_data[['assetCodes', 'assetName']].drop_duplicates(subset=['assetCodes', 'assetName'])
        # Split "assetCodes" from news_data
        code_name_news['assetCodes'] = code_name_news['assetCodes'].str.strip('{}').str.split(',')
        # For each assetCode in the list of "assetCodes", prepare a new row for each assetcode
        assetCode = code_name_news.apply(lambda x: pd.Series(x['assetCodes']), axis=1).stack().reset_index(level=1, drop=True)
        assetCode.name = 'assetCode'
        code_name_news.drop('assetCodes', axis=1, inplace=True)
        code_name_news_joined = code_name_news.join(assetCode).reset_index(drop=True)
        code_name_news_joined['assetCode'] = code_name_news_joined['assetCode'].apply(lambda x: x.replace("'",""))
        code_name_news_joined['assetCode'] = code_name_news_joined['assetCode'].apply(lambda x: x.replace(" ",""))
        # Rename assetName to assetName_news
        code_name_news_joined.rename(columns={'assetName': 'assetName_news'}, inplace=True)
        # Merge two dataframes together
        code_name_merged = pd.merge(left=code_name_market, right=code_name_news_joined, how='left', on=['assetCode'])
        # Filter out Unknown and NAN assetName
        code_name_merged.dropna(inplace=True)
        condition = code_name_merged['assetName'] == 'Unknown'
        code_name_filtered = code_name_merged[~condition]
        # Build up a dictionary to establish assetName mapping from news_data to market_data
        assetName_map = {}
        for index, row in code_name_filtered.iterrows():
            if row['assetName'] != row['assetName_news']:
                assetName_map[row['assetName_news']] = row['assetName']
        return assetName_map

    def merge_by_asset_name(self, market_data, news_data, assetName_map):
        """ Merge market_data and news_data, according to assetName
        """
        # Modify assetName in news_data according to assetName_map, so that it becomes the same as assetName in market_data
        news_data['assetName'] = news_data['assetName'].apply(lambda x: assetName_map[x] if x in assetName_map.keys() else x)
        news_data.drop(['assetCodes'], axis=1, inplace=True)
        # Integrate multiple news articles for a stock for a day into one aggregated row
        news_data_grouped = news_data.groupby(['time','assetName'], sort=False).aggregate(np.mean).reset_index()  
        news_data_grouped_max = news_data.groupby(['time','assetName'], sort=False).aggregate(np.max).reset_index()
        dummies = [column for column in news_data.columns if 'dummy_' in column]
        news_data_grouped[dummies] = news_data_grouped_max[dummies]
        del news_data, news_data_grouped_max
        # Merge two DataFrames
        return pd.merge(left=market_data, right= news_data_grouped, how='left', on=['time', 'assetName'], copy=False)
    
    def preprocess_and_merge(self):
        """ Preprocess market_data and news_data, and save the merged dataset
        """
        ### Initialization
        market_data = self.market_data
        news_data = self.news_data
        
        ### Preprocess news features
        # Convert subjects from texts to numeric dummies
        selected_subjects = ['ENER', 'BMAT', 'INDS', 'CYCS', 'NCYC', 'SHOP', 'FDRT', 'HECA', 'FINS', 'TECH', 'TCOM', 'MDIA', 
                             'UTIL', 'REAL', 'RES', 'RESF', 'RCH', 'DIV', 'IPO', 'MRG', 'DEAL1', 'BKRT', 'CEO1', 'LAYOFS',
                             'ACB', 'CLJ', 'BRIB', 'HACK', 'FAKE1', 'SCAM1', 'EMRG', 'US', 'WEU', 'ASIA', 'CN', 'STX', 'DBT',
                             'CDM', 'COM', 'DRV', 'FRX', 'GVD', 'HOT', 'MTG', 'PVE', 'MUNI', 'HEDGE']
        news_data['subjects'] = news_data['subjects'].apply(lambda x: 
                                                          [sub for sub in list(eval(x)) if sub in selected_subjects])
        mlb = MultiLabelBinarizer()
        dummy_subjects = pd.DataFrame(mlb.fit_transform(news_data['subjects']),
                                      columns=['dummy_' + name for name in mlb.classes_], index=news_data.index)
        selected_subjects = ['dummy_' + name for name in selected_subjects]
        news_data = pd.concat([news_data, dummy_subjects[selected_subjects]], axis = 1)
        del dummy_subjects
        print('Creating news data dummies: done!')
        # Drop some columns in news data (the list of dropped columns can be modified)
        drop_list = ['sourceTimestamp','firstCreated','sourceId','headline','urgency',
                     'provider','subjects','audiences','headlineTag','marketCommentary']
        news_data.drop(drop_list, axis=1, inplace=True)
        print('Drop columns for news data: done!')

        ### Preprocess market_data time
        market_data['time'] = pd.to_datetime(market_data['time']).apply(lambda x: x.date())
        print('Preprocessing market data time: done!')   

        ### Preprocess news_data time
        nytimezone = timezone("US/Eastern")
        # Convert from UTC to New York timezone
        news_data['time'] = pd.to_datetime(news_data['time']).apply(lambda x: x.tz_localize(UTC).astimezone(nytimezone))
        # Transform timestamp between time [9:30 am, date t ~ 9:30 am date t+1] to date t
        market_open = dt.time(9, 30, 0)
        news_data['time'] = news_data['time'].apply(lambda x: x.date() 
                                                    if x.time() >= market_open else x.date() - dt.timedelta(days = 1)) 
        ## Adjust 'time' for news_data: replace non-trading dates with last trading date (e.g. Sat/Sun to Fri)
        # First, get all the unique dates from news_data and market_data
        time_market = pd.DataFrame(market_data['time'].unique(), columns={'time'})
        time_news = pd.DataFrame(news_data['time'].unique(), columns={'time'})
        # Keep a copy of market_date before merging
        time_market['time_market'] = time_market['time']
        # Merge the two dataframes, the merged dataframe should have the same length with time_news
        # Also fill with the last trading date
        time_adjusted= pd.merge(left=time_market, right= time_news, how='right', on=['time'], sort=True).fillna(method='ffill')
        # Merge adjusted time to news_data
        news_data_adjusted = pd.merge(left=news_data, right=time_adjusted, how='left', on=['time'], copy=False)
        del news_data
        # Modify 'time_market' as the new 'time' column
        news_data_adjusted.drop(['time'], axis=1, inplace=True)
        news_data_adjusted.rename(columns={'time_market': 'time'}, inplace=True)
        print('Preprocessing news data time: done!')

        ### Find assetName map from market_data to news_data
        assetName_map = self.find_asset_name_map(market_data, news_data_adjusted)
        print('Find the assetName correspondences between market and news data: done!')

        ### Merge market and news data
        merged_data = self.merge_by_asset_name(market_data, news_data_adjusted, assetName_map)
        print('Merging market and news: done!')
        
        ### Save merged data
        merged_data.to_csv(self.out_filepath)
        print("Saving merged data: done!")

## 3. Call Class

In [6]:
start_time = time.time()
out_filepath = "../Data/merged_data.csv"
a = PreprocessAndMerge(market_df, news_df, out_filepath)
a.preprocess_and_merge()
print("Running time -- {0}".format(dt.timedelta(seconds = time.time() - start_time ) ) )

Creating news data dummies: done!
Drop columns for news data: done!
Preprocessing market data time: done!
Preprocessing news data time: done!
Find the assetName correspondences between market and news data: done!
Merging market and news: done!
Saving merged data: done!
Running time -- 0:29:53.267146
